In [1]:
!pip install langchain_community
!pip install langchain_chroma
!pip install langchain_huggingface
!pip install pypdf


Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/

In [2]:
import torch
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM, AutoTokenizer

# 初始化嵌入模型
def load_embedding_model():
    embedding_model_name = "/mnt/workspace/LLaMA-Factory/bge-large-en-v1.5"
    embedding_model_kwargs = {'device': 'cuda'}
    hf = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        model_kwargs=embedding_model_kwargs,
        encode_kwargs={'normalize_embeddings': True}
    )
    return hf

hf = load_embedding_model()

2025-04-08 15:40:07.376759: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-08 15:40:07.423654: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-08 15:40:08.438968: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# 加载生成模型
model_name = "/mnt/workspace/LLaMA-Factory/saves/Qwen2.5-7B-Instruct/lora/sft/checkpoint-336"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="cuda"
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
import os
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_pdf_and_split(path):
    loader = PyPDFLoader(path)
    pages = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=250,
        chunk_overlap=70,
        length_function=len,
        separators=["\n\n","\n" ,  " ",    ".",    ",",  ]
    )
    texts = text_splitter.split_documents(pages)
    return texts
def initialize_database(texts, persist_directory="Rag_file/chroma_db"):
    # 清空数据库
    if os.path.exists(persist_directory):
        db = Chroma(persist_directory=persist_directory, embedding_function=hf)
        db.delete_collection()

    # 创建新的数据库
    db = Chroma.from_documents(
        documents=texts,
        embedding=hf,
        persist_directory=persist_directory
    )
    return db
# def initialize_database(texts, persist_directory="Rag_file/chroma_db"):
#     if os.path.exists(persist_directory):
#         db = Chroma(persist_directory=persist_directory, embedding_function=hf)
#         existing_texts = db.get(include=["metadatas", "documents"])  # 获取现有文档和元数据
#         # 将现有文档重新构造为 Document 对象
#         existing_docs = [
#             Document(page_content=content, metadata=metadata)
#             for content, metadata in zip(existing_texts["documents"], existing_texts["metadatas"])
#         ]
#     else:
#         existing_docs = []

#     # 合并新旧文档
#     all_texts = existing_docs + texts

#     # 创建新的数据库
#     db = Chroma.from_documents(
#         documents=all_texts,
#         embedding=hf,
#         persist_directory=persist_directory
#     )
#     return db

# 主程序
path = "/mnt/workspace/LLaMA-Factory/Rag_file/assignment2.pdf"
# pdf_paths = [
#     "/mnt/workspace/LLaMA-Factory/Rag_file/assignment2.pdf",  # 原始文件
#     "/mnt/workspace/LLaMA-Factory/Rag_file/Certificate.pdf"   # 新文件
# ]
pages = load_pdf_and_split(path)
texts = load_pdf_and_split(path)
# all_texts = []
# for pdf_path in pdf_paths:
#     print(f"正在处理文件：{pdf_path}")
#     texts = load_pdf_and_split(pdf_path)
#     for text in texts:
#         text.metadata["file_name"] = os.path.basename(pdf_path)  # 添加文件名作为元数据
#     all_texts.extend(texts)

# 初始化或更新数据库
persist_directory = "Rag_file/chroma_db"
db = initialize_database(texts, persist_directory)

# 创建检索器
retriever = db.as_retriever(search_kwargs={"k": 10})


In [43]:
def generate_text(prompt, max_new_tokens=512):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.9,
            top_p=0.6,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

def format_response(response):
    # 移除多余内容
    response = response.split("回答：", 1)[-1].strip()
    
    # 格式化输出
    formatted_response = "\n".join([f"- {line.strip()}" for line in response.split("\n")])
    return formatted_response

# 定义问答函数
def answer_question(question):
    docs = retriever.invoke(question)
    context = "\n".join([doc.page_content for doc in docs])[:1024]  # 截断上下文长度
    prompt_template = f"""
    You are a helpful assistant. Based on the following context, answer the question in English
     上下文信息：
     {context}
    问题：
    {question}
    回答：
    """
    raw_answer = generate_text(prompt_template)
    clean_answer = clean_response(raw_answer)
    
    return clean_answer
# def answer_question(question):
#     docs = retriever.invoke(question)
#     context = "\n".join([doc.page_content for doc in docs])[:1024]  # 截断上下文长度
#     prompt_template = f"""
#     You are a helpful assistant. Based on the following context, answer the question in English
#     上下文信息：
#     {context}
#     问题：
#     {question}
#     回答：
#     """
#     answer = generate_text(prompt_template)
#     return answer





In [44]:
# 示例问题
question = "what is brief report's format"
response = answer_question(question)

# 打印结果
# print("问题：", question)
print( response)

问题：
    what is brief report's format
    回答：
     The brief report for this assignment should be submitted in PDF format and must not exceed 3 pages. It should clearly outline the method used in your analysis or project and provide an estimate of the number of games based on the results presented in your report. Additionally, you need to include a README.txt file that contains instructions on how to run your code. This report is crucial as it demonstrates your methodology and findings to the assessors. Ensure all information is concise and well-organized within the specified page limit.
